### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 28 мая 2022, 08:30   
**Штраф за опоздание:** по 1 баллу за 24 часа задержки.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0221, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier


In [2]:
class MyDecisionTreeClassifier:
    def __init__(self, min_samples_split=2, max_depth=5, criterion='gini'):
        """
        criterion -- критерий расщепления. необходимо релизовать три:
        Ошибка классификации, Индекс Джини, Энтропийный критерий
        max_depth -- максимальная глубина дерева
        min_samples_split -- минимальное число объектов в листе, чтобы сделать новый сплит
        """
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        # Для последнего задания
        self.feature_importances_ = None
        self.criterion = criterion
        # Структура, которая описывает дерево
        # Представляет словарь, где для  node_id (айдишник узла дерева) храним
        # (тип_узла, айдишник признака сплита, порог сплита) если тип NON_LEAF_TYPE
        # (тип_узла, предсказание класса, вероятность класса) если тип LEAF_TYPE
        # Подразумевается, что у каждого node_id в дереве слева
        # узел с айди 2 * node_id + 1, а справа 2 * node_id + 2
        self.tree = dict()

    @staticmethod
    def __div_samples(x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] < threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    @staticmethod
    def proba(y):
        labels = dict()
        for i in y:
            if i not in labels:
                labels[i] = 0
            labels[i] += 1
        len_label = len(labels)
        for i in labels:
            labels[i] /= len_label
        return labels

    @staticmethod
    def log_loss(labels):
        my_max = 0
        for key in labels:
            if labels[key] > my_max:
                my_max = labels[key]
        return 1 - my_max

    @staticmethod
    def gini(labels):
        my_sum = 0
        for key in labels:
            my_sum += labels[key]**2
        return 1 - my_sum

    @staticmethod
    def entropy(labels):
        my_sum = 0
        for key in labels:
            my_sum += labels[key] * math.log(labels[key], 2)
        return -my_sum

    def __find_threshold(self, x, y):
        """
        Находим оптимальный признак и порог для сплита
        Здесь используемые разные impurity в зависимости от self.criterion
        """
        my_min = np.inf
        len_y = len(y)
        for feature_id in range(x.shape[1]):
            for threshold in x[:, feature_id]:
                x_left, x_right, y_left, y_right = self.__div_samples(x, y, feature_id, threshold)
                if self.criterion == 'gini':
                    temp = (len(y_left)/len_y)*self.gini(self.proba(y_left))
                    temp += (len(y_right)/len_y)*self.gini(self.proba(y_right))
                elif self.criterion == 'entropy':
                    temp = (len(y_left)/len_y)*self.entropy(self.proba(y_left))
                    temp += (len(y_right)/len_y)*self.entropy(self.proba(y_right))
                elif self.criterion == 'log_loss':
                    temp = (len(y_left)/len_y)*self.log_loss(self.proba(y_left))
                    temp += (len(y_right)/len_y)*self.log_loss(self.proba(y_right))
                if temp < my_min:
                    my_min = temp
                    min_feature_id = feature_id
                    min_threshold = threshold
        return min_feature_id, min_threshold

    def __fit_node(self, x, y, node_id, depth):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с  айди 2 * node_id + 2
        """
        self.tree[node_id] = {'leaf': 0, 'depth': depth}
        probability = self.proba(y)
        if depth >= self.max_depth or len(y) < self.min_samples_split or len(probability) == 1:
            self.tree[node_id]['leaf'] = 1
            my_max = 0
            for key in probability:
                if probability[key] > my_max:
                    my_max = probability[key]
                    argmax = key
            self.tree[node_id]['leaf_predict'] = argmax
        if self.tree[node_id]['leaf'] == 0:
            feature_id, threshold = self.__find_threshold(x, y)
            self.tree[node_id]['feature_id'] = feature_id
            self.tree[node_id]['threshold'] = threshold
            x_left, x_right, y_left, y_right = self.__div_samples(x, y, feature_id, threshold)
            self.__fit_node(x_left, y_left, 2*node_id + 1, depth + 1)
            self.__fit_node(x_right, y_right, 2*node_id + 2, depth + 1)

    def fit(self, x, y):
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        node = self.tree[node_id]
        if node['leaf'] == 0:
            if x[node['feature_id']] < node['threshold']:
                return self.__predict_class(x, 2 * node_id + 1)
            return self.__predict_class(x, 2 * node_id + 2)
        return node['leaf_predict']

    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [3]:
my_clf = MyDecisionTreeClassifier()
clf = DecisionTreeClassifier()

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [5]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [6]:
print(clf.predict(X_test))
print(my_clf.predict(X_test))

[1 2 1 1 2 0 1 1 2 0 0 2 0 0 1 0 1 1]
[1 2 1 1 2 0 2 1 2 0 0 2 0 0 1 0 0 1]


In [7]:
print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))
print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.8333333333333334
0.9444444444444444


## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [8]:
%time clf.fit(X_train, y_train)

Wall time: 1.99 ms


DecisionTreeClassifier()

In [9]:
%time my_clf.fit(X_train, y_train)

Wall time: 458 ms
